# Spark session optimization



In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

## 1. Understand spark memory management

As we explained before, spark has different mode:
- local mode
- standalone
- yarn
- k8s

The memory management is quite different for each mode.

### 1.1 Local mode

In local mode, all Spark components (driver and executors) run inside **a single JVM process** on your machine. `unlike YARN or Kubernetes where resources are containerized and limited.`
A Spark driver/executor has two memory parts:
- JVM heap memory:
- OffHeap memory: stores `JVM metaspace, thread stacks, off-heap buffers (if enabled), native libs, Python/R processes in PySpark/SparkR`

```text
+------------------+        +-------------------+
| JVM Heap Memory  |        | Off-Heap Memory   |
|  - Objects       |        |  - Shuffle pages  |
|  - Small buffers |        |  - Column batches |
| GC managed       |        | Manual free()     |
+------------------+        +-------------------+
        ↑                            ↑
     Garbage                       Tungsten
     Collector                     MemoryMgr
```

#### 1.1.1 OffHeap memory size

By default, Spark sets :
- `spark.driver.memoryOverhead` to `max(384 MB, 0.10 * spark.driver.memory)`
- `spark.executor.memoryOverhead` to `max(384 MB, 0.10 * spark.executor.memory)`

For example, if I set `spark.driver.memory = 16GB`, then the default `overhead = 1.6GB`.

#### 1.1.2 Use OffHeap memory explicitly

By default, spark will use `Heap memory to store shuffle page and cache`. To work with large dataset, we need to allocate large JVM heaps. But Large JVM heaps (e.g., 20+ GB) cause **longer garbage collection pauses**. In this kind of situation, spark allows us to use OffHeap memory to store shuffle page and cache.


**Off-heap memory can help in cases:**
- Storing big data buffers (especially serialized blocks) off-heap: avoids longer garbage collection pauses
- Faster shuffle and caching: `Spark’s Tungsten engine` uses binary memory layouts that are well-suited for off-heap.
- Optimize size: Off-heap avoids object overhead (~16 bytes per object in heap).
- Columnar processing: Off-heap buffers align better with SIMD operations, making columnar execution (Arrow, Parquet) faster.


Below is an example of how to enable offHeap memory
```shell
# to enable spark to use offheap memory
spark.memory.offHeap.enabled = true
# when offheap enabled, the size must be set
spark.memory.offHeap.size = 4g

# we can also overwrite the default value of memory overhead
spark.driver.memoryOverhead = 2g
```

> The above config has a problem. Because the offHeap = 4g, and memoryOverHead = 2g. When a big join or groupby is executed, a OOM may happen.
> Because OffHeap is a part of the memoryOverhead. Best practice is memoryOverhead>=offHeap

#### 1.1.3 Spark driver memory architecture

```text
+--------------------  JVM Heap (spark.driver.memory / spark.executor.memory)
|  Reserved (JVM overhead: thread stacks, metaspace, GC)
|
|  Spark Unified Memory
|    +----------------- Execution Memory
|    |                 Storage Memory
|
+---------------------------------------------------------------------------------
   ↑ JVM-managed only

+--------------------  Memory Overhead (spark.driver/executor.memoryOverhead)
| Used for:
|    - JVM metaspace, thread stacks,
|    - native libs,
|    - Python/R processes in PySpark/SparkR
+--------------------  Off-Heap Memory (spark.memory.offHeap.size)
|  Managed by Spark Tungsten engine
|  Used for:
|    - Shuffle buffers
|    - Serialized cached blocks
|    - Columnar / Arrow / Parquet I/O
+---------------------------------------------------------------------------------
   ↑ OS-level RAM allocation (outside JVM heap)
```
#### 1.1.4 A memory config example

Suppose, we have a server with 32GB RAM running Spark in local mode, here’s an optimal memory sizing recommendation balancing JVM heap and off-heap:


- spark.driver.memory=20g: JVM heap for Spark driver. Big enough for dataset + tasks
- spark.memory.offHeap.enabled=true: Enable off-heap to reduce JVM GC pressure
- spark.memory.offHeap.size=6g: Off-heap buffers for shuffle, caching, serialization
- spark.driver.memoryOverhead=6g: Soft overhead reserve to cover off-heap and native libs

Why these values?
 - JVM heap (20 GB) + off-heap (6 GB) + OS & other processes (~6 GB) ≈ total 32 GB physical RAM.
 - Off-heap size (6 GB) lets Spark offload shuffle and caching buffers outside JVM heap, reducing GC pauses.
 - Leaves enough free RAM (~6 GB) for OS, background apps, and potential spikes.

#### 1.1.5 No memory limit shut down in local mode

### 1.2 Yarn and k8s mode

## 2. Unified memory management in spark

All memory reserved for the JVM heap is controlled by the spark unified memory management. It has three parts:
- JVM overhead
- memory for calculation
- memory for storage

### 2.1 Unified memory pool architecture


For example, when we define `spark.driver.memory=16GB`, it means we reserved `16GB for the JVM heap`. But spark can't use 16GB in reality, we need to remove `10% for the JVM overhead`.
Let's assume ~1.5GB overhead. So Spark can use `~14.5GB` in total. If we define `spark.memory.fraction = 0.7`(the default value is 0.6), it means spark will use 14.5GB*0.7=`10.15 for unified memory pool.`

Below shows the memory architecture with the config

```shell
spark.driver.memory=16GB
spark.memory.fraction=0.7
```

```text
JVM Heap 16GB
--- JVM Overhead (1.5 GB) – Non-heap
        ┌───────────────────────────────┐
        │ Threads, JIT, GC structures    │
        │ Direct buffers, native libs    │
        └───────────────────────────────┘
--- Spark (14.5 GB after overhead)
        ┌───────────────────────────────────────────┐
        │ Unified Memory Pool (~10.15 GB)            │
        │   ├── Execution Memory                     │
        │   └── Storage Memory                       │
        ├───────────────────────────────────────────┤
        │ User & Other Memory (~4.35 GB)             │
        │   ├── Python <-> JVM bridge objects        │
        │   ├── Spark plan metadata                  │
        │   ├── UDF intermediate data                │
        │   ├── Temporary parse buffers              │
        │   └── Non-managed Java objects             │
        └───────────────────────────────────────────┘


```

### 2.2 What lives outside the unified pool

Why do we reserve 30% memory(by default %40)? The rest 4.35GB is for:

- **Driver-side variables** (DataFrame/RDD metadata, Catalyst query plans, accumulators)
- **Task results & status updates** before they’re sent back
- **Broadcast variables on the driver** before being shipped to executors
- **Temporary JVM objects** from user code (e.g., Python → Java object conversions in PySpark)
- **UDF intermediate objects** (especially in Python UDFs with Arrow disabled)

If these 30% memory run out of space, you get `java.lang.OutOfMemoryError: Java heap space` even if Spark still shows **free** execution/storage memory in the UI.

> 0.7 is usually the safe upper limit in production. Going to 0.9 can be lethal unless the workload is extremely predictable.

In [2]:
spark = (
    SparkSession.builder
    .appName("LocalMode_memo_config")
    .master("local[*]")
    # JVM memory allocation
    .config("spark.driver.memory", "16g")  # Half of RAM for driver
    .config("spark.driver.maxResultSize", "4g")  # Avoid OOM on collect()
    # Shuffle & partition tuning
    .config("spark.sql.shuffle.partitions", "12")  # Lower than default 200
    .config("spark.sql.files.maxPartitionBytes", "128m")  # Avoid large partitions in memory
    .config("spark.reducer.maxSizeInFlight", "48m")  # Limit shuffle buffer
    # Unified memory management
    .config("spark.memory.fraction", "0.7")  # Reduce pressure on execution memory
    .config("spark.memory.storageFraction", "0.3")  # Smaller cache area
    .config("spark.memory.offHeap.enabled", "true")
    .config("spark.memory.offHeap.size", "1g")
    # Spill to disk early instead of crashing
    .config("spark.shuffle.spill", "true")
    .config("spark.shuffle.spill.compress", "true")
    .config("spark.shuffle.compress", "true")
    # optimize jvm GC
    .config("spark.driver.extraJavaOptions",
            "-XX:+UseG1GC -XX:InitiatingHeapOccupancyPercent=35 -XX:+HeapDumpOnOutOfMemoryError")
    # Use Kryo serializer
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    # Optional: buffer size for serialization
    .config("spark.kryoserializer.buffer", "64m")
    .config("spark.kryoserializer.buffer.max", "512m")
    .getOrCreate()
)

In [ ]:
# create a spark conf
conf = SparkConf()

conf.set("spark.master", "local[*]")  # Use all available cores
conf.set("spark.app.name", "OptimizedLocalSparkApp")

# MEMORY
conf.set("spark.driver.memory", "4g")                   # Heap memory
conf.set("spark.driver.memoryOverhead", "1024")          # Off-heap for native libs

# GC TUNING
conf.set("spark.executor.extraJavaOptions",
         "-XX:+UseG1GC -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent")

# OFF-HEAP (if using Arrow, Parquet, etc.)
conf.set("spark.memory.offHeap.enabled", "true")
conf.set("spark.memory.offHeap.size", "1g")

# SERIALIZATION
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")

# SHUFFLE OPTIMIZATION
conf.set("spark.shuffle.file.buffer", "1m")
conf.set("spark.reducer.maxSizeInFlight", "96m")
conf.set("spark.shuffle.io.preferDirectBufs", "true")

# PYTHON CONFIG
conf.set("spark.python.worker.memory", "2g")
conf.set("spark.pyspark.python", "C:/Users/PLIU/Documents/git/SparkInternals/si_venv/Scripts/python.exe")  # Replace with your Python path
conf.set("spark.pyspark.driver.python", "C:/Users/PLIU/Documents/git/SparkInternals/si_venv/Scripts/python.exe")

# OPTIONAL: avoid memory leak from large broadcast variables
conf.set("spark.cleaner.referenceTracking.blocking", "true")

spark = SparkSession.builder.config(conf=conf).getOrCreate()